In [11]:
# Instalando o pyspark
!pip install pyspark

In [12]:
# 2. Importanto bibliotecas do Spark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import re

In [14]:
if 'sc' in locals():
    sc.stop()

In [15]:
# Configuração do Spark
conf = SparkConf().setAppName("WordCount").setMaster("local[*]")
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

In [16]:
# Baixando o meu arquivo README.md da página da Compass do GitHub
!wget https://raw.githubusercontent.com/carolinacartaxo/compass-uol/main/README.md -O README.md


--2024-07-30 16:40:32--  https://raw.githubusercontent.com/carolinacartaxo/compass-uol/main/README.md
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2823 (2.8K) [text/plain]
Saving to: ‘README.md’

README.md           100%[===================>]   2.76K  --.-KB/s    in 0s      

2024-07-30 16:40:32 (16.0 MB/s) - ‘README.md’ saved [2823/2823]



In [17]:
# Criando uma função para remover pontuação e converter as palavras para minúsculas
def normalize_word(word):
    return re.sub(r'[^\w\s]', '', word).lower()  # Remove pontuações e converte para minúsculas

In [18]:
# Leitura do arquivo README.md usando Spark
# Aqui ele vai retornar um RDD em formato de string e não um data set
arquivo_texto = sc.textFile("README.md")

In [19]:
# Contagem das palavras
word_counts = arquivo_texto.flatMap(lambda line: line.split(" "))  # Divide cada linha em palavras

In [20]:
# Puxa a função criada anteriormente para normalizar as palavras
word_counts = word_counts.map(lambda word: normalize_word(word))

In [21]:
# Remove palavras vazias
word_counts = word_counts.filter(lambda word: word != "")

In [22]:
# Cria pares (palavra, 1) para poder somar a ocorrência de cada palavra
word_counts = word_counts.map(lambda word: (word, 1))

In [23]:
# Soma a contagens de cada palavra
word_counts = word_counts.reduceByKey(lambda a, b: a + b)

In [24]:
# 6. Converter o RDD para um DataFrame
word_counts_df = word_counts.toDF(["word", "count"])

In [25]:
# 7. Registrar o DataFrame como uma Tabela Temporária
word_counts_df.createOrReplaceTempView("word_counts")

In [26]:
# 8. Executar Consultas SQL para Verificar o Resultado
result_df = spark.sql("SELECT * FROM word_counts ORDER BY count DESC")
result_df.show()

+-----------+-----+
|       word|count|
+-----------+-----+
|         br|   44|
|         de|   20|
|          e|   19|
|     sprint|   14|
|        com|   12|
|        aws|   11|
|    desafio|    7|
|      dados|    6|
|       data|    6|
|     python|    6|
|      skill|    5|
|      curso|    5|
|       para|    5|
|    builder|    5|
|     básico|    4|
|    prática|    4|
|bibliotecas|    3|
|         da|    3|
|        uso|    3|
|         do|    3|
+-----------+-----+
only showing top 20 rows

